In [18]:
import mesa
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt

class FuzzyHerbivore(mesa.Agent):
    def __init__(self, model, hunger=0.5):
        super().__init__(model)
        self.hunger = hunger

    def fuzzy_decision(self):
        x_hunger = np.arange(0, 1.01, 0.01)
        hunger_low = fuzz.trimf(x_hunger, [0, 0, 0.5])
        hunger_med = fuzz.trimf(x_hunger, [0, 0.5, 1])
        hunger_high = fuzz.trimf(x_hunger, [0.5, 1, 1])

        membership_low = fuzz.interp_membership(x_hunger, hunger_low, self.hunger)
        membership_med = fuzz.interp_membership(x_hunger, hunger_med, self.hunger)
        membership_high = fuzz.interp_membership(x_hunger, hunger_high, self.hunger)

        memberships = {"eat": membership_high, "breed": membership_low, "move": membership_med}
        action = max(memberships, key=memberships.get)
        return action

    def step(self):
        decision = self.fuzzy_decision()
        # if decision != "move":
        print(f"Agent {self.unique_id} (hunger={self.hunger:.2f}) decides to {decision}.")
        if decision == "eat":
            self.hunger = max(self.hunger - 0.3, 0)
        else:
            self.hunger = min(self.hunger + 0.1, 1)

class SimpleModel(mesa.Model):
    def __init__(self, n, seed=None):
        super().__init__(seed=seed)
        self.num_agents = n

        FuzzyHerbivore.create_agents(model=self, n=n)

    def step(self):
        self.agents.shuffle_do("step")

In [19]:
starter_model = SimpleModel(100)

for s in range(1000):
    starter_model.step()